# 低次の多変量テイラー展開式を導出

In [10]:
import sympy as sy
from sympy import diff
from sympy.printing.numpy import NumPyPrinter

import original_kinematics

from pathlib import Path
import os

必要なシンボリック変数を宣言

In [11]:
N = 3  # テイラー展開の次数

# アクチュエータベクトル
l1, l2, l3 = sy.symbols("l1, l2, l3")
q = sy.Matrix([[l1, l2, l3]]).T

# ディスク位置を表すスカラ変数
xi = sy.Symbol("xi")


l1a, l2a, l3a = sy.symbols("l1a, l2a, l3a")  # 平衡点

テイラー展開する対象の式を導出

In [12]:
kinema = original_kinematics.Local()

P = kinema.P(q, xi)
R = kinema.R(q, xi)


テイラー展開式を導出

In [13]:
a, b, c = sy.symbols("a, b, c")
((a+b+c)**2).expand()

a**2 + 2*a*b + 2*a*c + b**2 + 2*b*c + c**2

In [14]:
((a+b+c)**3).expand()

a**3 + 3*a**2*b + 3*a**2*c + 3*a*b**2 + 6*a*b*c + 3*a*c**2 + b**3 + 3*b**2*c + 3*b*c**2 + c**3

テイラー展開（マクローリン展開）を実行

In [15]:

def multi_maclaurin_2(f):
    """a周りの2次の多変数テイラー展開"""
    f0 = f.subs([(l1, l1a), (l2, l2a), (l3, l3a)])
    
    f1 = diff(f, l1).subs([(l1, l1a), (l2, l2a), (l3, l3a)])*(l1-l1a) +\
        diff(f, l2).subs([(l1, l1a), (l2, l2a), (l3, l3a)])*(l2-l2a) +\
            diff(f, l3).subs([(l1, l1a), (l2, l2a), (l3, l3a)])*(l3-l3a)
    
    f2 = 1/2*(
        diff(diff(f, l1), l1).subs([(l1, l1a), (l2, l2a), (l3, l3a)])*(l1-l1a)**2 +\
            diff(diff(f, l2), l2).subs([(l1, l1a), (l2, l2a), (l3, l3a)])*(l2-l2a)**2 +\
                diff(diff(f, l3), l3).subs([(l1, l1a), (l2, l2a), (l3, l3a)])*(l3-l3a)**2 +\
                    2*diff(diff(f, l1), l2).subs([(l1, l1a), (l2, l2a), (l3, l3a)])*(l1-l1a)*(l2-l2a) +\
                        2*diff(diff(f, l2), l3).subs([(l1, l1a), (l2, l2a), (l3, l3a)])*(l2-l2a)*(l3-l3a) +\
                            2*diff(diff(f, l3), l1).subs([(l1, l1a), (l2, l2a), (l3, l3a)])*(l3-l3a)*(l1-l1a)
    )
    
    return f0 + f1 + f2



P_series = []
for i in range(3):
    P_series.append(
        multi_maclaurin_2(P[i, 0])
    )

R_series = [[]]*3
for i in range(3):
    for j in range(3):
        R_series[i].append(
            multi_maclaurin_2(R[i, j])
        )


In [16]:
# 微小値をlaに代入
for i in range(3):
    P_series[i] = P_series[i].subs([(l1a, 0.0001), (l2a, 0.002), (l3a, 0.0015)])

for i in range(3):
    for j in range(3):
        R_series[i][j] = R_series[i][j].subs([(l1a, 0.0001), (l2a, 0.002), (l3a, 0.0015)])

## 結果を出力

最後に自分で微小項や発散するを削除する必要あり

In [20]:
cwd = str(Path().resolve())
base = cwd + "/maclaurin_2"

dir_name = base
os.makedirs(dir_name, exist_ok=True)


numpy_word = "import numpy\ndef f(q, xi):\n    l1, l2, l3 = q[0,0], q[1,0], q[2,0]\n\n    return "

for i in range(3):
    f = open(dir_name + "/P_" + str(i) + ".py", 'w')
    f.write(numpy_word)
    f.write(NumPyPrinter().doprint(P_series[i]))
    f.close()

for i in range(3):
    for j in range(3):
        f = open(dir_name + "/R_" + str(i) + "_" + str(j) + ".py", 'w')
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(R_series[i][j]))
        f.close()

In [18]:
R_series[0][0]

0.5*(l1 - 0.0001)**2*(-6.07837243596557e-5*xi**2*cos(1.42156017576933e-5*xi) - 1.4723987711447*xi*sin(1.42156017576933e-5*xi) - 121438.163869571*cos(1.42156017576933e-5*xi) + 121438.163869571) + 0.5*(l1 - 0.0001)*(l2 - 0.002)*(8.8412689977681e-5*xi**2*cos(1.42156017576933e-5*xi) - 4.09059149510746*xi*sin(1.42156017576933e-5*xi) - 387780.362371481*cos(1.42156017576933e-5*xi) + 387780.362371481) + 0.5*(l1 - 0.0001)*(l3 - 0.0015)*(3.31547587416304e-5*xi**2*cos(1.42156017576933e-5*xi) + 7.03538903739686*xi*sin(1.42156017576933e-5*xi) + 630656.690110624*cos(1.42156017576933e-5*xi) - 630656.690110624) + (l1 - 0.0001)*(0.00754103754629469*xi*sin(1.42156017576933e-5*xi) - 73.0683388245297*cos(1.42156017576933e-5*xi) + 73.0683388245297) + 0.5*(l2 - 0.002)**2*(-3.21500690827931e-5*xi**2*cos(1.42156017576933e-5*xi) + 0.0899501794735671*xi*sin(1.42156017576933e-5*xi) - 133709.809902707*cos(1.42156017576933e-5*xi) + 133709.809902707) + 0.5*(l2 - 0.002)*(l3 - 0.0015)*(-2.41125518120948e-5*xi**2*cos(